In [1]:
import pandas as pd

import re
import requests
from tqdm.notebook import tqdm
import os

In [2]:
df = pd.read_csv("script_ref.csv", index_col=0)
df = df[df["script_downloaded"] == "Yes"]
df.reset_index(drop=True, inplace=True)

In [3]:
OMDB_columns = ["OMDB_Title", "Rating", "Release", "Runtime", "Genre", "Director", "Writer", 
                  "Language", "Country", "Noms", "IMDB", "IMDB_Votes", "Rotten_Tomatoes",
                  "Metascore", "Boxoffice"]

In [ ]:
OMDB = pd.DataFrame(index=df["title"], columns=OMDB_columns)

In [ ]:
for i in tqdm(range(950, len(OMDB)), desc="Processing"):

    title = df["title"][i]
    if ", The" in title:
        parts = title.split(", The")
        title = f"The {parts[0].rsplit(', ', 1)[-1]}"
    # Creating the API request url
    base_url = "http://www.omdbapi.com/"
    movie_id = "?t=" + title.replace(" ", "+")
    apikey = "&apikey=" + #API Key
    url = base_url + movie_id + apikey

    # Requesting the data and making a json file
    response = requests.get(url)
    data = response.json() 

    #Populating the dataframe with the appropriate values
    if data["Response"] == "True":
        OMDB["OMDB_Title"][i] = data["Title"]
        OMDB["Rating"][i] = data["Rated"]
        OMDB["Release"][i] = data["Released"]
        OMDB["Runtime"][i] = data["Runtime"]
        OMDB["Genre"][i] = data["Genre"]
        OMDB["Director"][i] = data["Director"]
        OMDB["Writer"][i] = data["Writer"]
        OMDB["Language"][i] = data["Language"]
        OMDB["Country"][i] = data["Country"]
        OMDB["Noms"][i] = data["Awards"]
        OMDB["IMDB"][i] = data["imdbRating"]
        OMDB["IMDB_Votes"][i] = data["imdbVotes"]
        OMDB["Rotten_Tomatoes"][i] = data["Ratings"][1]["Value"] if len(data["Ratings"]) >= 2 else None
        OMDB["Metascore"][i] = data["Metascore"]
        OMDB["Boxoffice"][i] = data["BoxOffice"] if "BoxOffice" in data else None

In [ ]:
OMDB.to_csv("OMDB_raw.csv")

Get Manual Coded Scipts Data

In [12]:
manual_codes = pd.read_csv("manual_omdb_codes.csv", header=None)

In [13]:
OMDB_manual = pd.DataFrame(index=manual_codes[0], columns=OMDB_columns)

In [14]:
for i in tqdm(range(len(OMDB_manual)), desc="Processing"):

    # Creating the API request url
    base_url = "http://www.omdbapi.com/"
    movie_id = "?i=" + manual_codes[1][i]
    apikey = "&apikey=" + "8e314a15"
    url = base_url + movie_id + apikey

    # Requesting the data and making a json file
    response = requests.get(url)
    data = response.json() 

    #Populating the dataframe with the appropriate values
    if data["Response"] == "True":
        OMDB_manual["OMDB_Title"][i] = data["Title"]
        OMDB_manual["Rating"][i] = data["Rated"]
        OMDB_manual["Release"][i] = data["Released"]
        OMDB_manual["Runtime"][i] = data["Runtime"]
        OMDB_manual["Genre"][i] = data["Genre"]
        OMDB_manual["Director"][i] = data["Director"]
        OMDB_manual["Writer"][i] = data["Writer"]
        OMDB_manual["Language"][i] = data["Language"]
        OMDB_manual["Country"][i] = data["Country"]
        OMDB_manual["Noms"][i] = data["Awards"]
        OMDB_manual["IMDB"][i] = data["imdbRating"]
        OMDB_manual["IMDB_Votes"][i] = data["imdbVotes"]
        OMDB_manual["Rotten_Tomatoes"][i] = data["Ratings"][1]["Value"] if len(data["Ratings"]) >= 2 else None
        OMDB_manual["Metascore"][i] = data["Metascore"]
        OMDB_manual["Boxoffice"][i] = data["BoxOffice"] if "BoxOffice" in data else None

Processing:   0%|          | 0/136 [00:00<?, ?it/s]

In [25]:
OMDB = pd.read_csv("OMDB_raw.csv", index_col=0)
OMDB.loc[OMDB.index.isin(OMDB_manual.index)] = OMDB_manual

Remove Empty Scripts

In [41]:
movie_data = pd.read_csv("allcorectOMDB.csv")

import os
for index, row in movie_data.iterrows():
    # Get the title and corresponding file name
    title = row['title']
    file_name = f"{title}.txt"
    
    # Construct the file path
    file_path = os.path.join("Scripts", file_name)
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Get the size of the file in KB
        file_size_kb = os.path.getsize(file_path) / 1024
        
        # Drop the row if the file size is less than 20 KB
        if file_size_kb < 20:
            print(f"Dropping title: {title}")
            movie_data.drop(index, inplace=True)

movie_data.reset_index(drop=True, inplace=True)



Dropping title: Apollo 13
Dropping title: Commando
Dropping title: Conan the Barbarian
Dropping title: eXistenZ
Dropping title: Frankenstein
Dropping title: Frankenweenie
Dropping title: Scary Movie 2
Dropping title: Things My Father Never Taught Me, The
Dropping title: This is 40


Cleaning Up Movie Data

In [46]:
movie_data = movie_data[movie_data["OMDB_Title"].isna() == False]
movie_data = movie_data.reset_index(drop=True)

In [50]:
movie_data['Release'] = pd.to_datetime(movie_data['Release'], format='%d %b %Y')

movie_data['Day_of_Year'] = movie_data['Release'].dt.dayofyear
movie_data['Week_of_Year'] = movie_data['Release'].dt.isocalendar().week
movie_data['Season'] = (movie_data['Release'].dt.month % 12 + 3) // 3
movie_data['Month'] = movie_data['Release'].dt.month
movie_data['Year'] = movie_data['Release'].dt.year

movie_data = movie_data.drop(columns="Release")

In [51]:
movie_data['Runtime'] = movie_data['Runtime'].str.extract('(\d+)').fillna(0).astype(int)

In [52]:
# Feature engenieering the total wins and total nominations data
movie_data['Total_Wins'] = movie_data['Noms'].str.extract('(\d+) wins?').astype(float)
movie_data['Total_Noms'] = movie_data['Noms'].str.extract('(\d+) nominations?').astype(float)

# Cleaning different website rating values
movie_data['IMDB'] = movie_data['IMDB'].astype(float)
movie_data['IMDB_Votes'] = movie_data['IMDB_Votes'].str.replace(',', '').astype(float)
movie_data.loc[~movie_data['Rotten_Tomatoes'].str.contains('%', na=False), 'Rotten_Tomatoes'] = 0
movie_data['Rotten_Tomatoes'] = movie_data['Rotten_Tomatoes'].str.replace('%', '').astype(float)
movie_data['Metascore'] = movie_data['Metascore'].astype(float)
movie_data['Boxoffice'] = movie_data['Boxoffice'].str.replace(',', '').str.replace('$', '').astype(float)

movie_data = movie_data.drop(columns="Noms")

movie_data.drop_duplicates(inplace=True)

/var/folders/g7/gnzs3l6x61xcl7ny9d10xjjm0000gn/T/ipykernel_24599/2784469213.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movie_data['Boxoffice'] = movie_data['Boxoffice'].str.replace(',', '').str.replace('$', '').astype(float)


In [56]:
movie_data.to_csv("movie_data_no_dummies.csv")

In [57]:
# Making dummy variables for Parent Rating
movie_data['Rating'].fillna('Unknown', inplace=True)
movie_data = movie_data.assign(Rating=movie_data['Rating'].str.split(', ')).explode('Rating')
Rating_indicators = pd.get_dummies(movie_data['Rating'], prefix='Rating')
movie_data = pd.concat([movie_data, Rating_indicators], axis=1)
movie_data.drop(columns=['Rating'], inplace=True)
Rating_columns = movie_data.columns[movie_data.columns.str.startswith('Rating')]
movie_data[Rating_columns] = movie_data.groupby(movie_data.index)[Rating_columns].transform('sum')
movie_data.drop_duplicates(inplace=True)

# Making dummy variables for Genres
movie_data['Genre'].fillna('Unknown', inplace=True)
movie_data = movie_data.assign(Genre=movie_data['Genre'].str.split(', ')).explode('Genre')
genre_indicators = pd.get_dummies(movie_data['Genre'], prefix='Genre')
movie_data = pd.concat([movie_data, genre_indicators], axis=1)
movie_data.drop(columns=['Genre'], inplace=True)
genre_columns = movie_data.columns[movie_data.columns.str.startswith('Genre')]
movie_data[genre_columns] = movie_data.groupby(movie_data.index)[genre_columns].transform('sum')
movie_data.drop_duplicates(inplace=True)

# Making dummy variables for Directors
movie_data['Director'].fillna('Unknown', inplace=True)
movie_data = movie_data.assign(Director=movie_data['Director'].str.split(', ')).explode('Director')
Director_indicators = pd.get_dummies(movie_data['Director'], prefix='Director')
movie_data = pd.concat([movie_data, Director_indicators], axis=1)
movie_data.drop(columns=['Director'], inplace=True)
Director_columns = movie_data.columns[movie_data.columns.str.startswith('Director')]
movie_data[Director_columns] = movie_data.groupby(movie_data.index)[Director_columns].transform('sum')
movie_data.drop_duplicates(inplace=True)

# Making dummy variables for Writers
movie_data['Writer'].fillna('Unknown', inplace=True)
movie_data = movie_data.assign(Writer=movie_data['Writer'].str.split(', ')).explode('Writer')
Writer_indicators = pd.get_dummies(movie_data['Writer'], prefix='Writer')
movie_data = pd.concat([movie_data, Writer_indicators], axis=1)
movie_data.drop(columns=['Writer'], inplace=True)
Writer_columns = movie_data.columns[movie_data.columns.str.startswith('Writer')]
movie_data[Writer_columns] = movie_data.groupby(movie_data.index)[Writer_columns].transform('sum')
movie_data.drop_duplicates(inplace=True)

# Making dummy variables for Languages
movie_data['Language'].fillna('Unknown', inplace=True)
movie_data = movie_data.assign(Language=movie_data['Language'].str.split(', ')).explode('Language')
Language_indicators = pd.get_dummies(movie_data['Language'], prefix='Language')
movie_data = pd.concat([movie_data, Language_indicators], axis=1)
movie_data.drop(columns=['Language'], inplace=True)
Language_columns = movie_data.columns[movie_data.columns.str.startswith('Language')]
movie_data[Language_columns] = movie_data.groupby(movie_data.index)[Language_columns].transform('sum')
movie_data.drop_duplicates(inplace=True)

# Making dummy variables for Countries
movie_data['Country'].fillna('Unknown', inplace=True)
movie_data = movie_data.assign(Country=movie_data['Country'].str.split(', ')).explode('Country')
Country_indicators = pd.get_dummies(movie_data['Country'], prefix='Country')
movie_data = pd.concat([movie_data, Country_indicators], axis=1)
movie_data.drop(columns=['Country'], inplace=True)
Country_columns = movie_data.columns[movie_data.columns.str.startswith('Country')]
movie_data[Country_columns] = movie_data.groupby(movie_data.index)[Country_columns].transform('sum')
movie_data.drop_duplicates(inplace=True)

In [58]:
movie_data.to_csv("movie_data_dummies.csv")